<a href="https://colab.research.google.com/github/Naomie25/DI-Bootcamp/blob/main/Week8_Day3_lesson1_tuto_ROUGE_BLEU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to implement BLEU in Python ?

Begin by importing the required modules:

We import sqrt, log, and exp to perform fundamental mathematical operations—square-root extraction, natural logarithms, and exponentiation—and Counter to efficiently count and tally elements in iterable collections.

In [1]:
from math import sqrt, log, exp
from collections import Counter

We define the hypothesis and reference texts to establish the candidate output and its ground-truth comparisons, using examples drawn from David Bamman’s NLP23 repository.


In [2]:
hypothesis="Abandon all hope , ye who enter here"
references=["All hope abandon , ye who enter here", "All hope abandon , ye who enter in !", "Leave every hope, ye that enter", "Leave all hope , ye that enter"]


Then, we have to get the n-grams from the given text. We’ll define a function to get the frequency of n-grams from the given text of size “order”.


In [3]:
#Getting the n-grams from the given text
def get_ngrams(text, order):
    """
    Given a string `text` and an integer `order`, returns a Counter object containing
    the frequency counts of all ngrams of size `order` in the string.
    """
    ngrams = Counter()

    words = text.split()
    for i in range(len(words)- order+1):
      ngram = " ". join(words[i: i + order])
      ngrams[ngram] += 1

    return ngrams

🚀 Challenge : Print the n-grams!

Now, let’s calculate bleu, which is done in four steps as clearly indicated in the code block below:

In [4]:
def calculate_bleu(hypothesis, references):

    bleu=0
    p1=0
    p2=0
    p3=0
    p4=0
    bp=1

    # 1. Find the closest reference to the hypothesis
    closest_size=100000
    closest_ref=[]

    for ref in references:
      ref_size = len(ref)
      if abs(len(hypothesis) - ref_size) < closest_size:
        closest_size = abs(len(hypothesis) - ref_size)
        closest_ref = ref
        pass

    # 2. Calculating pn
    pns=[]
    for order in range(1,5):
      # calculate intersection and union of n-grams
      # hint: use the get_ngrams function you implemented
      # calculate pn for each order
        hyp_ngrams = get_ngrams(hypothesis, order)
        hyp_count = Counter(hyp_ngrams)
        closest_ref_ngrams = get_ngrams(closest_ref, order)
        closest_ref_count = Counter(closest_ref_ngrams)
        intersection_count = dict(hyp_count & closest_ref_count)
        intersection_size = sum(intersection_count.values())
        hyp_size = max(len(hyp_ngrams), 1)
        p_n = intersection_size / hyp_size
        pns.append(p_n)
        pass

    # 3. Calculating the brevity penalty
    bp=1
    c=len(hypothesis)
    r=min(abs(len(ref) - c) for ref in references)
    if c > r:
      bp = 1.0
    else:
      bp = exp(1 - r / c)

    # 4. Calculating the BLEU score
    weights = [0.25] * 4
    bleu=bp * exp(sum(w * log(p_n) for w, p_n in zip(weights, pns)))

    # Assigning values to p1, p2, p3, p4!
    p1, p2, p3, p4 = pns


    # Do not change the variable name
    return bleu, p1, p2, p3, p4, bp

Final step? Calculate and print the bleu score. The value should be 0.5<bleu<1.

In [5]:
bleu, p1, p2, p3, p4, bp=calculate_bleu(hypothesis, references)
print("BLEU: %.3f" % bleu)

BLEU: 0.541


🚀 Challenge : What was the bleu value ? What does it indicate?

# How to implement it in Python : Computing ROUGE Scores in Python

We will follow a step-by-step tutorial for computing ROUGE-N and ROUGE-L metrics. You can also follow this [colab](https://colab.research.google.com/drive/1yU9Hwg356TdLUONB4ozuQ72H9DauXQLh)

#### 1. Imports and Example Texts

We’ll reuse `Counter` for n-grams and implement our own LCS routine:

In [6]:
from collections import Counter

hypothesis = "Abandon all hope , ye who enter here"
references = [
    "All hope abandon , ye who enter here",
    "All hope abandon , ye who enter in !",
    "Leave every hope, ye that enter",
    "Leave all hope , ye that enter"
]

#### 2. N-gram Extraction

In [7]:
def get_ngrams(text, order):
    """
    Returns a Counter of all n-grams of size `order` in `text`.
    """
    ngrams = Counter()
    words = text.split()
    for i in range(len(words) - order + 1):
        ngram = " ".join(words[i : i + order])
        ngrams[ngram] += 1
    return ngrams

🚀 **Challenge**: Print the 1- and 2-grams of the hypothesis!

#### 3. ROUGE-N (Recall, Precision, F₁)

For each n (e.g., 1, 2), compute:

* **Recall** = overlap\_ngrams / total\_ngrams\_in\_reference
* **Precision** = overlap\_ngrams / total\_ngrams\_in\_hypothesis
* **F₁** = 2 × P × R / (P + R)

Average over references by taking the maximum overlap against any single reference.

In [8]:
def rouge_n(hyp, refs, n):
    """
    Compute ROUGE-N (recall, precision, f1) for one hypothesis vs. multiple references.
    """
    hyp_ngrams = get_ngrams(hyp, n)
    best = {"overlap": 0, "ref_count": 0}

    for ref in refs:
        ref_ngrams = get_ngrams(ref, n)
        overlap = sum((hyp_ngrams & ref_ngrams).values())
        if overlap > best["overlap"]:
            best["overlap"] = overlap
            best["ref_count"] = sum(ref_ngrams.values())

    hyp_count = sum(hyp_ngrams.values())
    recall    = best["overlap"] / best["ref_count"] if best["ref_count"] > 0 else 0.0
    precision = best["overlap"] / hyp_count         if hyp_count > 0        else 0.0
    f1 = (2 * precision * recall / (precision + recall)
          if (precision + recall) > 0 else 0.0)

    return recall, precision, f1

#### 4. ROUGE-L (Longest Common Subsequence)

ROUGE-L uses the LCS length between hypothesis and reference:

* **Recall** = LCS / |reference|
* **Precision** = LCS / |hypothesis|
* **F₁** = (1 + β²)·P·R / (R + β²·P), with β = 1 by default.


In [9]:
def _lcs_length(a, b):
    """Compute length of LCS between sequences a and b via dynamic programming."""
    m, n = len(a), len(b)
    dp = [[0]*(n+1) for _ in range(m+1)]
    for i in range(m):
        for j in range(n):
            if a[i] == b[j]:
                dp[i+1][j+1] = dp[i][j] + 1
            else:
                dp[i+1][j+1] = max(dp[i][j+1], dp[i+1][j])
    return dp[m][n]


def rouge_l(hyp, refs, beta=1.0):
    """
    Compute ROUGE-L (recall, precision, f1) for one hypothesis vs. multiple references.
    Takes the reference yielding the highest F1.
    """
    best = {"f1": 0, "r": 0, "p": 0}
    hyp_tokens = hyp.split()

    for ref in refs:
        ref_tokens = ref.split()
        lcs = _lcs_length(hyp_tokens, ref_tokens)
        r = lcs / len(ref_tokens) if ref_tokens else 0.0
        p = lcs / len(hyp_tokens)   if hyp_tokens else 0.0
        denom = r + (beta**2) * p
        f1 = ((1 + beta**2) * p * r / denom) if denom > 0 else 0.0

        if f1 > best["f1"]:
            best.update({"f1": f1, "r": r, "p": p})

    return best["r"], best["p"], best["f1"]

Compute and print ROUGE-1, ROUGE-2, and ROUGE-L on our example:

In [10]:
# ROUGE-1
r1, p1, f1 = rouge_n(hypothesis, references, 1)
print(f"ROUGE-1 → recall: {r1:.3f}, precision: {p1:.3f}, F1: {f1:.3f}")

# ROUGE-2
r2, p2, f2 = rouge_n(hypothesis, references, 2)
print(f"ROUGE-2 → recall: {r2:.3f}, precision: {p2:.3f}, F1: {f2:.3f}")

# ROUGE-L
rl_r, rl_p, rl_f1 = rouge_l(hypothesis, references)
print(f"ROUGE-L → recall: {rl_r:.3f}, precision: {rl_p:.3f}, F1: {rl_f1:.3f}")

ROUGE-1 → recall: 0.750, precision: 0.750, F1: 0.750
ROUGE-2 → recall: 0.571, precision: 0.571, F1: 0.571
ROUGE-L → recall: 0.750, precision: 0.750, F1: 0.750
